In [1]:
1+1

2

In [50]:
import json
import numpy as np
import pandas as pd

In [29]:
raw_df = pd.read_csv("/Users/jacsonchong/Documents/GitHub/worldquant-capstone/python_code/data/l2_data.csv")

In [30]:
raw_df.head()

,ts_recv_ns,ws_idx,raw
0,1758544902394491000,0,"{""channel"":""l2_data"";""client_id"":"""";""timestamp..."
1,1758544902465620000,0,"{""channel"":""l2_data"";""client_id"":"""";""timestamp..."
2,1758544902465646000,0,"{""channel"":""l2_data"";""client_id"":"""";""timestamp..."
3,1758544902465665000,0,"{""channel"":""l2_data"";""client_id"":"""";""timestamp..."
4,1758544902465685000,0,"{""channel"":""l2_data"";""client_id"":"""";""timestamp..."


In [17]:
df['raw'].values.tolist()[-1]

'{"channel":"l2_data";"client_id":"";"timestamp":"2025-09-22T15:08:56.000212391Z";"sequence_num":1824;"events":[{"type":"update";"product_id":"SOL-USD";"updates":[{"side":"bid";"event_time":"2025-09-22T15:08:55.975863Z";"price_level":"221.01";"new_quantity":"49.11194588"};{"side":"bid";"event_time":"2025-09-22T15:08:55.975863Z";"price_level":"221";"new_quantity":"73.78440537"};{"side":"bid";"event_time":"2025-09-22T15:08:55.975863Z";"price_level":"220.94";"new_quantity":"101.55732926"};{"side":"bid";"event_time":"2025-09-22T15:08:55.975863Z";"price_level":"220.86";"new_quantity":"283.81246928"};{"side":"bid";"event_time":"2025-09-22T15:08:55.975863Z";"price_level":"220.79";"new_quantity":"25.00324007"};{"side":"bid";"event_time":"2025-09-22T15:08:55.975863Z";"price_level":"193.4";"new_quantity":"0.04061942"};{"side":"offer";"event_time":"2025-09-22T15:08:55.975863Z";"price_level":"221.11";"new_quantity":"102.45586586"};{"side":"offer";"event_time":"2025-09-22T15:08:55.975863Z";"price_l

In [42]:
def parse_l2_message(raw):
    fixed = raw.replace(";", ",")
    data = json.loads(fixed)
    
    timestamp = data.get("timestamp")
    sequence_num = data.get("sequence_num")
    events = data.get("events", [])

    parsed_records = []

    for event in events:
        event_type = event.get("type")
        product_id = event.get("product_id")
        
        if event_type == "snapshot":
            updates = event.get("updates", [])
            for u in updates:
                # Handle both "quantity" and "new_quantity" just in case
                quantity = u.get("quantity") or u.get("new_quantity")
                parsed_records.append({
                    "event_type": event_type,
                    "side": u.get("side"),
                    "price_level": float(u.get("price_level")),
                    "quantity": float(quantity),
                    "event_time": u.get("event_time"),
                    "product_id": product_id,
                    "sequence_num": sequence_num,
                    "timestamp": timestamp,
                })

        elif event_type == "update":
            updates = event.get("updates", [])
            for u in updates:
                parsed_records.append({
                    "side": u["side"],
                    "price_level": float(u["price_level"]),
                    "new_quantity": float(u["new_quantity"]),
                    "event_time": u.get("event_time"),
                    "product_id": product_id,
                    "sequence_num": sequence_num,
                    "timestamp": timestamp,
                    "event_type": event_type
                })
    
    # Convert to DataFrame
    if parsed_records:
        return pd.DataFrame(parsed_records)
    else:
        return pd.DataFrame()

In [43]:
parsed_list = raw_df["raw"].apply(parse_l2_message)

# Concatenate all parsed DataFrames
final_df = pd.concat(parsed_list.tolist(), ignore_index=True)

In [44]:
final_df

,event_type,side,price_level,quantity,event_time,product_id,sequence_num,timestamp,new_quantity
0,snapshot,bid,112614.81,0.044387,2025-09-22T12:41:41.049849763Z,BTC-USD,0,2025-09-22T12:41:41.458968292Z,NaN
1,snapshot,bid,112612.73,0.000067,2025-09-22T12:41:41.049849763Z,BTC-USD,0,2025-09-22T12:41:41.458968292Z,NaN
2,snapshot,bid,112612.72,0.001776,2025-09-22T12:41:41.049849763Z,BTC-USD,0,2025-09-22T12:41:41.458968292Z,NaN
3,snapshot,bid,112612.65,0.006047,2025-09-22T12:41:41.049849763Z,BTC-USD,0,2025-09-22T12:41:41.458968292Z,NaN
4,snapshot,bid,112612.62,0.174131,2025-09-22T12:41:41.049849763Z,BTC-USD,0,2025-09-22T12:41:41.458968292Z,NaN
...,...,...,...,...,...,...,...,...,...
226140,update,bid,193.40,NaN,2025-09-22T15:08:55.975863Z,SOL-USD,1824,2025-09-22T15:08:56.000212391Z,0.040619
226141,update,offer,221.11,NaN,2025-09-22T15:08:55.975863Z,SOL-USD,1824,2025-09-22T15:08:56.000212391Z,102.455866
226142,update,offer,221.12,NaN,2025-09-22T15:08:55.975863Z,SOL-USD,1824,2025-09-22T15:08:56.000212391Z,57.335206
226143,update,offer,221.17,NaN,2025-09-22T15:08:55.975863Z,SOL-USD,1824,2025-09-22T15:08:56.000212391Z,165.587706


In [45]:
final_df['product_id'].unique()

array(['BTC-USD', 'ETH-USD', 'SOL-USD', 'ADA-USD'], dtype=object)

In [52]:
def build_event_lists(df, product_id, side_map=("bid", "ask")):
    df_prod = df[df["product_id"] == product_id].copy()
    df_prod["dt"] = pd.to_datetime(df_prod["event_time"].fillna(df_prod["timestamp"]))
    t0 = df_prod["dt"].min() # the earliest time
    df_prod["t"] = (df_prod["dt"] - t0).dt.total_seconds().astype(float)
    event_lists = []
    for side in side_map:
        times = df_prod[df_prod["side"] == side]["t"].sort_values().to_numpy()
        event_lists.append(times)
    T = df_prod["t"].max()
    return event_lists, float(T)

In [53]:
def negative_loglikelihood_exp(params, event_lists, T, d):
    
    p = np.exp(params) # map parameters (all positive) via exponential transform 
    mu = p[:d]
    A = p[d:d + d * d].reshape((d, d))  # A[i,j] effect on i from component j
    omega = p[-1]

    N = [len(ev) for ev in event_lists] # Create counts per component and flattened list of all events (for intensity computation)

    # Compute log-likelihood
    eps = 1e-12
    loglik = 0.0

    integral = np.zeros(d)
    for i in range(d):
        integral[i] = mu[i] * T
        for j in range(d):
            if N[j] == 0:
                continue
            tj = event_lists[j]
            integral[i] += (A[i, j] / omega) * np.sum(1.0 - np.exp(-omega * (T - tj)))
    for i in range(d):
        ti = event_lists[i]
        for t_n in ti:
            lam = mu[i]
            for j in range(d):
                tj = event_lists[j]
                if len(tj) == 0:
                    continue
                idx = np.searchsorted(tj, t_n, side='left')
                if idx > 0:
                    diffs = t_n - tj[:idx]
                    lam += np.sum(A[i, j] * np.exp(-omega * diffs))
            if lam <= 0:
                loglik += -1e6
            else:
                loglik += np.log(lam + eps)
    total_ll = loglik - np.sum(integral)
    return -total_ll

In [54]:
from scipy.optimize import minimize

def estimate_hawkes_exp(event_lists, T, verbose=True):
    d = len(event_lists)

    total_events = sum(len(ev) for ev in event_lists) # initial guesses
    if total_events == 0:
        raise ValueError("No events to estimate from.")
    # crude starting guesses:
    mu0 = np.array([max(1e-4, len(ev) / (T+1e-9) * 0.5) for ev in event_lists])  # baseline
    A0 = np.eye(d) * 0.2  # small self-excitation parameter
    A0 = A0 + 0.05  # small cross terms parameter
    omega0 = 1.0
    param0 = np.log(np.concatenate([mu0, A0.flatten(), np.array([omega0])]))

    res = minimize(
        fun=negative_loglikelihood_exp,
        x0=param0, # baseline params
        args=(event_lists, T, d),
        method='L-BFGS-B', # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
        options={'disp': verbose, 'maxiter': 500}
    )
    if not res.success and verbose:
        print("Optimization warning:", res.message)
    p_hat = np.exp(res.x)
    mu_hat = p_hat[:d]
    A_hat = p_hat[d:d + d * d].reshape((d, d))
    omega_hat = p_hat[-1]
    return {"mu": mu_hat, "A": A_hat, "omega": omega_hat, "neg_loglikehood": res.fun, "success": res.success, "message": res.message}


In [55]:
event_lists, T = build_event_lists(final_df, product_id="BTC-USD", side_map=("bid", "ask"))

In [62]:
res = estimate_hawkes_exp(event_lists, T)
print("Estimated mu:", res["mu"])
print("Estimated A:\n", res["A"])
print("Estimated omega:", res["omega"])
print("Neg loglikelihood:", res["neg_loglikehood"])

/var/folders/gd/8rykv2pd50b10tqjb80ntm3h0000gn/T/ipykernel_12175/4268679199.py:16: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  res = minimize(


Estimated mu: [6.09057966e+00 7.86935969e-08]
Estimated A:
 [[8.54686480e-003 5.00000000e-002]
 [2.57862001e-256 2.50000000e-001]]
Estimated omega: 0.06691914991316258
Neg loglik: -69595.06123901258
